In [1]:
import numpy as np

In [2]:
A=[
    [4,3,0,5,0],
    [5,0,4,4,0],
    [4,0,5,0,3],
    [2,3,0,1,0],
    [0,4,2,0,5]
]

In [4]:
A=np.array(A)

In [6]:
A.shape

(5, 5)

In [13]:
def data_processing(x):
    data=[]
    for i in range(x.shape[0]):
        temp=[]
        for j in range(x.shape[1]):
            if x[i][j]==0:
                temp.append(0)
            else:
                temp.append(1)
        data.append(temp)
    return data

In [14]:
data_processing(A)

[[1, 1, 0, 1, 0],
 [1, 0, 1, 1, 0],
 [1, 0, 1, 0, 1],
 [1, 1, 0, 1, 0],
 [0, 1, 1, 0, 1]]

In [15]:
def generate_dict(dataTmp):
	'''将用户-商品矩阵转换成二部图的表示
	input:  dataTmp(mat):用户商品矩阵
	output: data_dict(dict):图的表示
	'''
	m, n = np.shape(dataTmp)

	data_dict = {}
	# 对每一个用户生成节点
	for i in range(m):
		tmp_dict = {}
		for j in range(n):
			if dataTmp[i, j] != 0:
				tmp_dict["D_" + str(j)] = dataTmp[i, j]
		data_dict["U_" + str(i)] = tmp_dict

	# 对每一个商品生成节点
	for j in range(n):
		tmp_dict = {}
		for i in range(m):
			if dataTmp[i, j] != 0:
				tmp_dict["U_" + str(i)] = dataTmp[i, j]
		data_dict["D_" + str(j)] = tmp_dict
	return data_dict

In [16]:
def PersonalRank(data_dict, alpha, user, maxCycles):
	'''利用PersonalRank打分
	input:  data_dict(dict):用户-商品的二部图表示
			alpha(float):概率
			user(string):指定用户
			maxCycles(int):最大的迭代次数
	output: rank(dict):打分的列表
	'''
	# 1、初始化打分
	rank = {}
	for x in data_dict.keys():
		rank[x] = 0
	rank[user] = 1  # 从user开始游走

	# 2、迭代
	step = 0
	while step < maxCycles:
		tmp = {}
		for x in data_dict.keys():
			tmp[x] = 0

		for i, ri in data_dict.items():
			for j in ri.keys():
				if j not in tmp:
					tmp[j] = 0
				tmp[j] += alpha * rank[i] / (1.0 * len(ri))
				if j == user:
					tmp[j] += (1 - alpha)
		# 判断是否收敛
		check = []
		for k in tmp.keys():
			check.append(tmp[k] - rank[k])
		if sum(check) <= 0.0001:
			break
		rank = tmp
		if step % 20 == 0:
			print(
			"iter: ", step)
		step = step + 1
	return rank

In [20]:
datamat=np.mat(data_processing(A))

In [21]:
data_dict=generate_dict(datamat)

In [24]:
PersonalRank(data_dict, 0.85, "U_0", 1000)

iter:  0
iter:  20
iter:  40


{'D_0': 0.40555832841159833,
 'D_1': 0.3419589631501934,
 'D_2': 0.1711298436120938,
 'D_3': 0.35482680386602944,
 'D_4': 0.10449676372091332,
 'U_0': 0.7336363128884573,
 'U_1': 0.2352347290192902,
 'U_2': 0.17911159250522568,
 'U_3': 0.28363631288845725,
 'U_4': 0.18981882061031582}

In [26]:
rank=PersonalRank(data_dict, 0.85, "U_0", 1000)

iter:  0
iter:  20
iter:  40


In [25]:

def recommend(data_dict, rank, user):
	'''得到最终的推荐列表
	input:  data_dict(dict):用户-商品的二部图表示
			rank(dict):打分的结果
			user(string):用户
	output: result(dict):推荐结果
	'''
	items_dict = {}
	# 1、用户user已打过分的项
	items = []
	for k in data_dict[user].keys():
		items.append(k)

	# 2、从rank取出商品的打分
	for k in rank.keys():
		if k.startswith("D_"):  # 商品
			if k not in items:  # 排除已经互动过的商品
				items_dict[k] = rank[k]

	# 3、按打分的降序排序
	result = sorted(items_dict.items(), key=lambda d: d[1], reverse=True)
	return result

In [27]:
recommend(data_dict,rank,"U_0")

[('D_2', 0.1711298436120938), ('D_4', 0.10449676372091332)]